# Import Libraries

In [142]:
import os              # looking for files under a folder
import pandas as pd    # dataframe 
from tqdm import tqdm  # to get progress bar
import re              # to get start and end position of string

# To convert spacy NER data to IOB BERT NER data format
import spacy 
from spacy.training import offsets_to_biluo_tags

In [2]:
os.getcwd()

'C:\\Users\\ajaya\\github_my_refernce'

# Input Variables to be declared

In [41]:
## Input var
#1. #OCRed (Image and pdf), 2. text_plain file/ Email(For email file it needs to be coverted to txt)/Excel(need to be converted to text) 
# For now only for text files #########need to think about email and excel
root_data_path = "C:\\Users\\ajaya\\github_my_refernce\\data\\NER_data\\OCRed_txt_files\\"
entity_file_excel = r'C:\Users\ajaya\github_my_refernce\data\NER_data\ner_entity_labels.xlsx' # filename and each entity in each column and value as string
sheet_name = 'Sheet1'
entities = ['person','geo','post','mo_no'] # Make sure these names are same as entity file column names





# Read files under a folder and read it with output as dataframe

In [51]:
# read file
def get_file_content(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    
    #lines are in list form one element as one list
    #join it to get all test in one line
    one_line = ' '.join(lines)
    
    return one_line
    
## function to get details of files and its content in one df
def read_root_txt_files(root_data_path):
    cols = ['full_file_path', 'file_name']#,'content']
    op_df = pd.DataFrame(columns = cols)
    
    # r=root, d=directories, f = files
    for r, d, f in tqdm(os.walk(root_data_path)):
        for file in f:
            #if file.endswith(".docx"):
            full_file_path = os.path.join(r, file)
            print(full_file_path)
            
            op_df = op_df.append({'full_file_path': full_file_path, 
                                  'file_name': file,
                                  'content': get_file_content(full_file_path)
                                 }, ignore_index=True)
    
    return op_df

In [52]:
read_df = read_root_txt_files(root_data_path)
read_df

1it [00:00, 52.60it/s]

C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\1.txt
C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\2.txt
C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\3.txt
C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\4.txt
C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\5.txt
C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\6.txt
C:\Users\ajaya\github_my_refernce\data\NER_data\OCRed_txt_files\7.txt


,full_file_path,file_name,content
0,C:\Users\ajaya\github_my_refernce\data\NER_dat...,1.txt,Ajay is from the city of Pune. From last 2 yea...
1,C:\Users\ajaya\github_my_refernce\data\NER_dat...,2.txt,Vijay is from the city of Pune. From last 2 ye...
2,C:\Users\ajaya\github_my_refernce\data\NER_dat...,3.txt,Mohan is from the city of Pune. From last 2 ye...
3,C:\Users\ajaya\github_my_refernce\data\NER_dat...,4.txt,Mohan is from the city of Pune. From last 2 ye...
4,C:\Users\ajaya\github_my_refernce\data\NER_dat...,5.txt,Mohan is from the city of kolkattA. From last ...
5,C:\Users\ajaya\github_my_refernce\data\NER_dat...,6.txt,Suku is from the city of kolkattA. From last 2...
6,C:\Users\ajaya\github_my_refernce\data\NER_dat...,7.txt,Muku is from the city of kolkattA. From last 2...


# Read tagged entity file

In [53]:
tag_entity_df = pd.read_excel(entity_file_excel, sheet_name=sheet_name)

In [54]:
tag_entity_df.head(10)

,file_name,person,geo,post,mo_no
0,1.txt,ajay,bangalore,data science,9.574875e+09
1,2.txt,vijay,NaN,analyst,NaN
2,3.txt,mohan,NaN,devops,NaN
3,4.txt,NaN,pune,Power BI Developer,1.457847e+09
4,5.txt,NaN,kolkatta,NaN,NaN
5,6.txt,suku,NaN,java developer,NaN
6,7.txt,muku,mumbai,NaN,2.144563e+09


# Create NER data input format for Spacy
https://1.bp.blogspot.com/-VVOd5rDWsEQ/XhTSleznIUI/AAAAAAAAB6Y/z_ZNyIiwjL4Fzshqa4e6-1YeKR2zvatiQCLcBGAsYHQ/s1600/image002.jpg

## Join files and entity dataframes

In [104]:
df_files_entity = read_df.merge(tag_entity_df, on='file_name', how='left')
df_files_entity.head(10)

,full_file_path,file_name,content,person,geo,post,mo_no
0,C:\Users\ajaya\github_my_refernce\data\NER_dat...,1.txt,Ajay is from the city of Pune. From last 2 yea...,ajay,bangalore,data science,9.574875e+09
1,C:\Users\ajaya\github_my_refernce\data\NER_dat...,2.txt,Vijay is from the city of Pune. From last 2 ye...,vijay,NaN,analyst,NaN
2,C:\Users\ajaya\github_my_refernce\data\NER_dat...,3.txt,Mohan is from the city of Pune. From last 2 ye...,mohan,NaN,devops,NaN
3,C:\Users\ajaya\github_my_refernce\data\NER_dat...,4.txt,Mohan is from the city of Pune. From last 2 ye...,NaN,pune,Power BI Developer,1.457847e+09
4,C:\Users\ajaya\github_my_refernce\data\NER_dat...,5.txt,Mohan is from the city of kolkattA. From last ...,NaN,kolkatta,NaN,NaN
5,C:\Users\ajaya\github_my_refernce\data\NER_dat...,6.txt,Suku is from the city of kolkattA. From last 2...,suku,NaN,java developer,NaN
6,C:\Users\ajaya\github_my_refernce\data\NER_dat...,7.txt,Muku is from the city of kolkattA. From last 2...,muku,mumbai,NaN,2.144563e+09


In [111]:
df_files_entity['mo_no'] = df_files_entity['mo_no'].fillna(0)
df_files_entity['mo_no'] = df_files_entity['mo_no'].astype('int')
df_files_entity.head()

,full_file_path,file_name,content,person,geo,post,mo_no
0,C:\Users\ajaya\github_my_refernce\data\NER_dat...,1.txt,Ajay is from the city of Pune. From last 2 yea...,ajay,bangalore,data science,-2147483648
1,C:\Users\ajaya\github_my_refernce\data\NER_dat...,2.txt,Vijay is from the city of Pune. From last 2 ye...,vijay,NaN,analyst,0
2,C:\Users\ajaya\github_my_refernce\data\NER_dat...,3.txt,Mohan is from the city of Pune. From last 2 ye...,mohan,NaN,devops,0
3,C:\Users\ajaya\github_my_refernce\data\NER_dat...,4.txt,Mohan is from the city of Pune. From last 2 ye...,NaN,pune,Power BI Developer,1457846872
4,C:\Users\ajaya\github_my_refernce\data\NER_dat...,5.txt,Mohan is from the city of kolkattA. From last ...,NaN,kolkatta,NaN,0


In [105]:
df_files_entity['content'][0]

'Ajay is from the city of Pune. From last 2 years he has shifted to bangalore city.bangalore is really good city and there are not pythons here.I request Ajay A to shift to pune if possible. He is a data science person.There is really high demand in market of data science. I hope every one will get in touch with me soon on the 9574875143 this number. This is not my personal number so dont call. Hahaha\n \n'

## Search entities in text_content and get start and end position

In [106]:
def get_search_entity(text, pattern, entity_name):
    entity_dict = {}
    for match in re.finditer(pattern, text):
        s = match.start()
        e = match.end()
        #print ('String match "',s1[s:e],'" at', s,':', e)
        print(s,e,entity_name)
        

In [73]:
text = df_files_entity['content'][0].lower()
pattern = df_files_entity['post'][0].lower()
entity_name = 'post'

get_search_entity(text, pattern, entity_name)

198 210 post
259 271 post


In [107]:
len(df_files_entity)

7

## Create spacy format

In [130]:
# Prepare spacy formatted training data
def spacy_data_format(data, ents, txt_col):
    TRAIN_DATA = []
    ent_list = []
    for row in range(len(data)):
        #print(row)
        ent_list_sen = []
        for entity_name in ents:
            #print(entity_name)
            pattern = str(data[entity_name][row]).lower()
            for match in re.finditer(pattern, data[txt_col][row].lower()):
                s = match.start()
                e = match.end()
                ent_list_sen.append(((s),(e),entity_name))
        ent_list.append(ent_list_sen)
        ## Create blank dictionary
        ent_dic = {}
        ## Fill value to the dictionary
        ent_dic['entities'] = ent_list_sen
        ## Prepare final training data
        TRAIN_DATA.append((data[txt_col][row],ent_dic))
        
    return TRAIN_DATA

In [133]:
spacy_ner_data = spacy_data_format(df_files_entity, entities, txt_col = 'content')
spacy_ner_data

[('Ajay is from the city of Pune. From last 2 years he has shifted to bangalore city.bangalore is really good city and there are not pythons here.I request Ajay A to shift to pune if possible. He is a data science person.There is really high demand in market of data science. I hope every one will get in touch with me soon on the 9574875143 this number. This is not my personal number so dont call. Hahaha\n \n',
  {'entities': [(0, 4, 'person'),
    (153, 157, 'person'),
    (67, 76, 'geo'),
    (82, 91, 'geo'),
    (198, 210, 'post'),
    (259, 271, 'post')]}),
 ('Vijay is from the city of Pune. From last 2 years he has shifted to bangalore city.bangalore is really good city and there are not pythons here.I request Ajay A to shift to pune if possible. He is a data analyst person.There is really high demand in market of data science. I hope every one will get in touch with me soon on the this number. This is not my personal number so dont call. Hahaha\n \n',
  {'entities': [(0, 5, 'perso

# Convert Spacy NER data to IOB / BERT NER data format

In [185]:
# Refernece: https://stackoverflow.com/questions/59200123/converting-spacy-training-data-format-to-spacy-cli-format-for-blank-ner/59209377#59209377

# python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm') 
docs = []
for text, annot in spacy_ner_data:
    doc = nlp(text)
    tags = spacy.training.offsets_to_biluo_tags(doc, annot['entities'])
    entities = spacy.training.biluo_tags_to_spans(doc, tags)
    # Convert L->I and U->B to have IOB tags for the tokens in the doc
    
    doc.ents = entities
    docs.append(doc)

tag_data = spacy.training.docs_to_json(docs[0:2])

print(tag_data) ## Check output on http://jsonviewer.stack.hu/

{'id': 0, 'paragraphs': [{'raw': 'Ajay is from the city of Pune. From last 2 years he has shifted to bangalore city.bangalore is really good city and there are not pythons here.I request Ajay A to shift to pune if possible. He is a data science person.There is really high demand in market of data science. I hope every one will get in touch with me soon on the 9574875143 this number. This is not my personal number so dont call. Hahaha\n \n', 'sentences': [{'tokens': [{'id': 0, 'orth': 'Ajay', 'space': ' ', 'tag': 'NNP', 'pos': 'PROPN', 'morph': 'Number=Sing', 'lemma': 'Ajay', 'head': 1, 'dep': 'nsubj', 'ner': 'U-person'}, {'id': 1, 'orth': 'is', 'space': ' ', 'tag': 'VBZ', 'pos': 'AUX', 'morph': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin', 'lemma': 'be', 'head': 0, 'dep': 'ROOT', 'ner': 'O'}, {'id': 2, 'orth': 'from', 'space': ' ', 'tag': 'IN', 'pos': 'ADP', 'morph': '', 'lemma': 'from', 'head': -1, 'dep': 'prep', 'ner': 'O'}, {'id': 3, 'orth': 'the', 'space': ' ', 'tag': 'D

In [186]:
tag_data['id']

0

In [188]:
print(len(tag_data))
print(tag_data['paragraphs'][0]['sentences'][0]['tokens'][0]['orth'])
print(tag_data['paragraphs'][0]['sentences'][0]['tokens'][0]['ner'])

2
Ajay
U-person


In [204]:

str1 = 'U-person'
str1.replace("U-","B-")
str1

'U-person'

In [208]:
def spacy_to_iob_format(spacy_data):
    # Refernece: https://stackoverflow.com/questions/59200123/converting-spacy-training-data-format-to-spacy-cli-format-for-blank-ner/59209377#59209377

    # python -m spacy download en_core_web_sm
    nlp = spacy.load('en_core_web_sm') 
    docs = []
    for text, annot in spacy_data:
        doc = nlp(text)
        tags = spacy.training.offsets_to_biluo_tags(doc, annot['entities'])
        entities = spacy.training.biluo_tags_to_spans(doc, tags)
        # Convert L->I and U->B to have IOB tags for the tokens in the doc

        doc.ents = entities
        docs.append(doc)

    tag_data = spacy.training.docs_to_json(docs)
    
    # Create empty dataframe for final iob output
    df_op_iob = pd.DataFrame()
    
    for i_para in range(len(tag_data['paragraphs'])):
        for i_sent in range(len(tag_data['paragraphs'][i_para]['sentences'])):
            for i_tok in range(len(tag_data['paragraphs'][i_para]['sentences'][i_sent]['tokens'])):
                doc_id = i_para # each document is one paragraphs
                word = tag_data['paragraphs'][i_para]['sentences'][i_sent]['tokens'][i_tok]['orth']
                tag = tag_data['paragraphs'][i_para]['sentences'][i_sent]['tokens'][i_tok]['ner']
                # Convert L->I and U->B to have IOB tags for the tokens in the doc
                tag = tag.replace("L-","I-")
                tag = tag.replace("U-","B-")
                #print(i_para,'>>',word, '>>', tag)
                df_op_iob = df_op_iob.append({'doc_id': doc_id, 
                                              'word': word,
                                              'ner_tag': tag
                                             }, ignore_index=True) 
    
    return df_op_iob

In [216]:
iob_ner_data = spacy_to_iob_format(spacy_ner_data)
iob_ner_data

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Ajay is from the city of Pune. From last 2 years h..." with entities "[(0, 4, 'person'), (153, 157, 'person'), (67, 76, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Mohan is from the city of kolkattA. From last 2 ye..." with entities "[(26, 34, 'geo'), (178, 186, 'geo')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,doc_id,ner_tag,word
0,0.0,B-person,Ajay
1,0.0,O,is
2,0.0,O,from
3,0.0,O,the
4,0.0,O,city
...,...,...,...
608,6.0,O,do
609,6.0,O,nt
610,6.0,O,call
611,6.0,O,.


In [218]:
iob_ner_data[iob_ner_data['word']=='Mohan']

,doc_id,ner_tag,word
175,2.0,B-person,Mohan
262,3.0,O,Mohan
350,4.0,O,Mohan


In [211]:
iob_ner_data['ner_tag'].value_counts()

O           587
B-person      8
B-post        6
B-geo         5
I-post        5
B-mo_no       2
Name: ner_tag, dtype: int64